In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Under sampling in Python

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

#scaler=StandardScaler()
#x_train= scaler.fit_transform(x_train0)
#x_test= scaler.transform(x_test)
x_train= x_train0.copy()
x_test = x_test.copy()

# Data normalization (Scaling choice)

In [ ]:
%%script false

# conclusion use RobustScaler

from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy=0.01,random_state=42)
x_train, y_train = undersample.fit_resample(x_train0, y_train0)

modelClf = KNeighborsClassifier()

modelClf = KNeighborsClassifier()
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

f1=fd.calculate_scores(y_train, predsTrain,'f1')
print("f1 train {:.4f}".format(f1))
f1=fd.calculate_scores(y_test, predsTest,'f1')
print("f1 test  {:.4f}".format(f1))

scalers = fd.getScalers()
for key in scalers:
    print(key)
    modelClf = KNeighborsClassifier()
    scaler=scalers.get(key)
    x_train2= scaler.fit_transform(x_train) 
    x_test2= scaler.transform(x_test)
    x_train2 = pd.DataFrame(x_train2, columns=[predictors])
    x_test2 = pd.DataFrame(x_test2, columns=[predictors])

    modelClf = KNeighborsClassifier()
    modelClf.fit(x_train2, y_train)
    predsTrain = modelClf.predict(x_train2)
    predsTest = modelClf.predict(x_test2)

    f1=fd.calculate_scores(y_train, predsTrain,'f1')
    print("f1 train {:.4f}".format(f1))
    f1=fd.calculate_scores(y_test, predsTest,'f1')
    print("f1 test  {:.4f}".format(f1))
    


In [ ]:
%%script false
# found rate = 0.01

from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

rates =[0.99,0.9,0.5,0.25,0.2, 0.1,0.05, 0.01, 0.005]
for rate in rates:
    print(rate)
    undersample = RandomUnderSampler(sampling_strategy=rate,random_state=42)
    x_train, y_train = undersample.fit_resample(x_train0, y_train0)
    scaler = RobustScaler()
    x_train2= scaler.fit_transform(x_train) 
    x_test2= scaler.transform(x_test)
    x_train2 = pd.DataFrame(x_train2, columns=[predictors])
    x_test2 = pd.DataFrame(x_test2, columns=[predictors])
    print(x_train.shape)
    
    modelClf = KNeighborsClassifier()
    modelClf.fit(x_train2, y_train)
    predsTrain = modelClf.predict(x_train2)
    predsTest = modelClf.predict(x_test2)

    f1=fd.calculate_scores(y_train, predsTrain,'f1')
    print("f1 train {:.4f}".format(f1))
    f1=fd.calculate_scores(y_test, predsTest,'f1')
    print("f1 test  {:.4f}".format(f1))


In [ ]:
%%script false
# found n_neighbors=6

import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler

undersample = RandomUnderSampler(sampling_strategy=0.01,random_state=42)
x_train, y_train = undersample.fit_resample(x_train0, y_train0)

scaler = RobustScaler()
x_train2= scaler.fit_transform(x_train) 
x_test2= scaler.transform(x_test)
x_train2 = pd.DataFrame(x_train2, columns=[predictors])
x_test2 = pd.DataFrame(x_test2, columns=[predictors])

arange =np.arange(1,11,1)
for n_neighbors in arange:
    print('n_neighbors',n_neighbors)
    print(x_train.shape)    
    modelClf = KNeighborsClassifier(n_neighbors=n_neighbors)
    modelClf.fit(x_train2, y_train)
    predsTrain = modelClf.predict(x_train2)
    predsTest = modelClf.predict(x_test2)

    f1=fd.calculate_scores(y_train, predsTrain,'f1')
    roc=fd.calculate_scores(y_train, predsTrain,'roc_auc_score')
    print("f1  train {:.4f}".format(f1))
    print("roc train {:.4f}".format(roc))
    f1=fd.calculate_scores(y_test, predsTest,'f1')
    roc=fd.calculate_scores(y_test, predsTest,'roc_auc_score')
    print("f1  test  {:.4f}".format(f1))
    print("roc test  {:.4f}".format(roc))

# Final validation

In [ ]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler

undersample = RandomUnderSampler(sampling_strategy=0.01,random_state=42)
x_train, y_train = undersample.fit_resample(x_train0, y_train0)
print(x_train.shape)  

scaler = RobustScaler()
x_train2= scaler.fit_transform(x_train) 
x_test2= scaler.transform(x_test)
x_train2 = pd.DataFrame(x_train2, columns=[predictors])
x_test2 = pd.DataFrame(x_test2, columns=[predictors])

modelClf = KNeighborsClassifier(n_neighbors=6)
modelClf.fit(x_train2, y_train)
predsTrain = modelClf.predict(x_train2)
predsTest = modelClf.predict(x_test2)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test2,y_test)

In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)
    y=dfTrx['Class']

    x_train2= scaler.fit_transform(dfTrx[predictors]) 
    dfTrx = pd.DataFrame(x_train2, columns=[predictors])
    

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(y, preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(y, preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('05-sklearn.neighbors','KNeighborsClassifier','2-After-tuning-V2', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])
